# **Instituto Tecnológico Vale**

## **Introdução a Visão Computacional**

### **Reconhecimento de imagem**

## **Exemplo prático**

### Construção e treinamento de CNN para classificação de imagens

In [ ]:
# baixar e descompactar os dados do exemplo
# Será criado a pasta hymenoptera_data no ambiente do colab

!wget -O hymenoptera_data.zip https://www.dropbox.com/s/154i61alh62e7jl/hymenoptera_data.zip?dl=0;

!unzip hymenoptera_data.zip;

In [ ]:
#imports necessários

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Criando os datasets e dataloaders

# Aqui usamos a função ImageFolder para criar um dataset deireto das pastas organizadas train e val.
# O pytorch entende automaticamente os nomes das pastas ants e bees como as classes.

data_dir = 'hymenoptera_data'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'Dataset sizes: {dataset_sizes}')
print(f'class names: {class_names}')
print(f'cuda: {device}')

In [ ]:
# Funções auxiliares para a prática

# Função auxiliar para impimir um batch de imagens

def imshow(inp, title=None):
    """Imshow for Tensor."""

    #plt.figure(figsize=(10, 8))

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated



# Function to test output dimension
def out_conv2d(dim_input, kernel_size, padding=0, dilation=1, stride=1):
    dim_output = ((dim_input + 2 * padding - dilation * (kernel_size-1) - 1)/stride) + 1
    return dim_output

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
# função de treino

def train_model(model, criterion, optimizer, num_epochs=15):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Função a ser chamada após treino para visualizar a classificação de um batch

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# testando as saidas da camadas conv2d
# Use esta opção para clacular corretamente as dimensões

# Insira a dimensão de entrada e o tamanho do filtro
out_conv2d(111, 3)

In [ ]:
# Criando uma rede convolucional

class classificador(nn.Module):

  def __init__(self):
    super().__init__()

    # Criação das camadas convolucionais
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3)) # saida 222x222 (no pooling cai pela metade -> 111)
    self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3)) #saida 109x109 (no pooling cai pela metade -> 54)

    # Criação da função relu
    self.activation = nn.ReLU()

    # Criação do pooling
    self.pool = nn.MaxPool2d(kernel_size = (2,2))

    # Criação do flatten para vetorzar a imagem ao final das camadas conv
    self.flatten = nn.Flatten() # 54*54 * 128 channels

    # Camadas lineares da rede neural
    self.linear1 = nn.Linear(in_features=128 * 54*54, out_features=128)
    self.linear2 = nn.Linear(128, 64)
    self.output = nn.Linear(64, 2)

    # Dropout para diminuir overfitting
    self.dropout = nn.Dropout(p = 0.2)

  # Fluxo da passagem da imagem na rede 
  def forward(self, X):
    X = self.pool(self.activation(self.conv1(X)))
    X = self.pool(self.activation(self.conv2(X)))
    
    X = self.flatten(X)

    X = self.dropout(self.activation(self.linear1(X)))
    X = self.dropout(self.activation(self.linear2(X)))
    X = self.output(X)

    return X

In [ ]:
# Criando objeto da estrutura da rede

net = classificador()
print(net)

In [ ]:
# Definindo parâmetros importantes do treinamento

#Função de custo e função de otimização dos parâmetros

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [ ]:
# Colocar a rede na gpu
net.to(device)

In [ ]:
# treinar modelo
trained_model = train_model(net, criterion, optimizer, num_epochs=15)

In [ ]:
# Visualizar imagens do conjunto de teste classificadas
visualize_model(trained_model)

### Redes Neurais convolucionais pré treinadas e Transfer Learning

Existem modelos pre treinados disponiveis no modulo models do pytorch.

Estas redes foram treinadas e validadas em competições e são capzaes de classificar várias coisas: 1000 classes.

Podemos usar estas redes como base para nossos problemas e retreiná-las com nossos dados.



In [ ]:
# Carregando do pytorch a rede chamada vgg16
model_ft = models.vgg16(pretrained=True)

In [ ]:
print(model_ft)

In [ ]:
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_ft.classifier[6].in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier[6] = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001)

In [ ]:
print(model_ft)

In [ ]:
# treinar modelo
trained_model = train_model(model_ft, criterion, optimizer_ft, num_epochs=15)